In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import skimage.io
from tqdm.notebook import tqdm
import random
import cv2
import seaborn as sns
import os
# plt.rcParams['figure.figsize'] = [15,8]

In [ ]:
DATA_DIR = '/kaggle/input/global-wheat-detection/'
train_csv = pd.read_csv(DATA_DIR + 'train.csv')
train_csv.head()

bounding box values = (value 1) = topleft x cooordinate of box (value 2) = topleft y cooordinate of box (value 3) = width of box (value 4) = height of box

train_csv['width'] = width of image train_csv['height'] = height of image

In [ ]:
import re
def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r

train_csv['x'] = -1.0
train_csv['y'] = -1.0
train_csv['w'] = -1.0
train_csv['h'] = -1.0

train_csv[['x', 'y', 'w', 'h']] = np.stack(train_csv['bbox'].apply(lambda x: expand_bbox(x)))

train_csv['x'] = train_csv['x'].astype(np.float)
train_csv['y'] = train_csv['y'].astype(np.float)
train_csv['w'] = train_csv['w'].astype(np.float)
train_csv['h'] = train_csv['h'].astype(np.float)

Input argument format for opencv rectangle function = ( (topleft x coordinate, topleft y coordinate), (bottomright x coordinate, bottomright y coordinate) bordor colour, border width) = ( (train_csv[x], train_csv[y]), (train_csv[x]+train_csv[w], train_csv[y]+train_csv[h]), red, 2)

In [ ]:
import cv2
N=147793
random_index = random.randint(0,N)
img_path = DATA_DIR + 'train/' + '2c7241d19' + '.jpg'
img = cv2.imread(img_path)

image_id = train_csv.iloc[random_index]['image_id']
image_id='2c7241d19'
boxes = train_csv[train_csv['image_id'] == image_id][['x', 'y', 'w', 'h']].values

fig, ax = plt.subplots(1, 1, figsize=(16, 8))
for box in boxes:
    xmin = np.int(box[0])
    ymin = np.int(box[1])
    width = np.int(box[2])
    height = np.int(box[3])
    img = cv2.rectangle(img,(xmin,ymin),(xmin+width,ymin+height),(255,0,0),2)
ax.set_axis_off()
ax.imshow(img)

Plan:

Train training set images on a pretrained model of YOLOv3 or miniYOLO

To do the same we need to first prepare the dataset in the format that the model accepts. This link clearly explains how we could do this: https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects

Our first step in this process would be to convert our bounding box representation. Current representation = (x, y, w, h) = (topleft x coordinate, topleft y coordinate, width of bounding box, height of bounding box)

Required representation = (yolox, yoloy, yolow, yoloh) = (centre x coordinate of bounding box w.r.t image width, centre y coordinate of bounding box w.r.t image height, width of bounding box w.r.t image width, height of bounding box w.r.t image height) = ( ((x + (w/2))/width, ((y + (h/2))/height, w/width, h/height )

In [ ]:
train_csv['yolox'] = (train_csv['x'] + (train_csv['w']/2))/train_csv['width']
train_csv['yoloy'] = (train_csv['y'] + (train_csv['h']/2))/train_csv['height']
train_csv['yolow'] = (train_csv['w'])/train_csv['width']
train_csv['yoloh'] = (train_csv['h'])/train_csv['height']
train_csv['class'] = 0

train_csv.head()


Not able to run the YOLO model in kaggle as the github folder is in the /input folder and that folder is read only in Kaggle.

Couldn't train the model in personal laptop as GPU is not CUDA supported and we need a GPU with CUDA support (usually NVIDIA GPUs) to train this model. So decided to train the model in Google colab and then upload the trained weights to kaggle.
Here is the link to the code in Google colab to train the model. (wheat-detection-yolo.ipynb).
https://colab.research.google.com/drive/1MdD-8JSQO0CyVj9sXqhBVQLC9hPAspok

The following changes needed to be made to the prepared dataset/github folder to train the model:
1. set GPU=1, CUDNN=1, OPENCV=1 in the Makefile under the /darknet folder (the root folder)
2. Had to change the following values in the build/darknet/x64/cfg/yolo-obj.cfg file:
    a. subdivisions = 64
    b. height = 512
    c. width = 512
    Decreasing the height and width values from 1024(the image height and width) to 512 could decrease the quality in object detection but the GPU memory wasn't enough, so had to decrease. Maybe there is a way to increase the GPU memory in Google colab.
3. Had to change all the paths in build/darknet/x64/data/obj.data file to build/darknet/x64/data
4. Had to chane all paths in build/darknet/x64/data/train.txt file to build/darknet/x64/
5. Had to add the file paths of images with no object present in the build/darknet/x64/data/train.txt file as I had forgotten to add them earlier.
6. Had to change all the paths in the command to train the model from:
./darknet detector train data/obj.data yolo-obj.cfg yolov4.conv.137 to 
!./darknet detector train build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/yolov4.conv.137 -dont_show
7. To train the model, the above command was run in the darknet folder that is the root folder /
8. Added the -dont_show option in the above command as it was causing errors while training the model, to display certain results.

To run the model in Google colab, additionally the following steps were needed:
1. Add the kaggle key (json) file to Google colab.
2. Download the dataset, using the API command for the dataset in Kaggle and unzip it: kaggle datasets download -d sneha5gsm/yolowheatdetection
3. Download dos2unix package to convert the (config, data, txt) files to Unix format.
4. Initially wasn't able to train the model in Google colab, the notebook used to crash, so created the following issue in Github for that repo, the suggestions provided by @waelausy solved the issue:
https://github.com/AlexeyAB/darknet/issues/5554#issuecomment-626318806



Next steps:
1. Get the weights after training the model in Google colab and test it out.
2. If the results seem ok, upload the weights into kaggle.
3. Copy the logs generated in Google colab to analyse cost details, etc.
4. See if a different pretrained weight, or model gives better results.
5. Browse how different parameter values in Makefile, config file can affect the results.

Next steps:
1. Got weights after training the model for almost 12 hours in Google Colab. Could get upto 2000 iterations.
2. Testing train images and random images from internet with the weights and network is giving pretty impressive results.
3. Try to run the detect command in kaggle. Currently I can think of two options:
    a. Upload the customized darknet folder to github and clone it in Kaggle.
    b. Try to figure out a way to use the uploaded zip folder in kaggle in the working directory section, rather than input folder section.   --- this seems like a better option as of now
4. Convert the output bounding box result into csvs.
5. Generate a submission.csv file for the test images in the dataset.
6. Submit the submission and check out the score.

Finally, We cannot use internet in this competition. So, created a zip with the darknet executable, uploaded it as a dataset. Copying it from input directory to working directory and then run the detect cmd.

Best results(weights) after training have been uploaded in this notebook for testing and submission.

In [ ]:
!cp -r /kaggle/input/yolowheatdetectionnoopencv /kaggle/working

In [ ]:
#turn on GPU to run this
os.chdir("/kaggle/working/yolowheatdetectionnoopencv/darknet-master")
!chmod +x ./darknet
# !./darknet detector test build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/yolo-obj_2000.weights

In [ ]:
# %%capture cap
import fileinput
import re
import glob

test_image_ids = []
for dirname, _, filenames in os.walk('/kaggle/input/global-wheat-detection/test/'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        test_image_ids.append(filename)

PAT = 'wheathead:'
FILES = 'output.txt'
values = []
for image_id in test_image_ids:
    image_path = '/kaggle/input/global-wheat-detection/test/' + image_id
#     print(image_path)
    output = !./darknet detector test build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/yolo-obj_2000.weights -ext_output $image_path -dont_show
    boxes = ''
    for line in output:
        if re.search(PAT, line):
            arrtemp = line.split()
            left_x = int(arrtemp[3])
            if left_x<0:
                left_x = 0
            top_y = int(arrtemp[5])
            if top_y<0:
                top_y = 0
            boxes = boxes + str(int(arrtemp[1][:-1])/100) + ' ' + str(left_x) + ' ' + str(top_y) + ' ' + str(arrtemp[7]) + ' ' + str(arrtemp[9][:-1]) + ' '
    boxes = boxes[:-1]
    values.append([image_id[:-4], boxes])
sample_submission = pd.DataFrame(values, columns=['image_id', 'PredictionString'])
sample_submission.to_csv('submission.csv', index=False)
# !cp  submission.csv /kaggle/working
    


In [ ]:
# PAT = 'wheathead:'
# FILES = 'output.txt'
# image_id = '00b70a919.jpg'
# image_path = '/kaggle/input/global-wheat-detection/train/' + image_id
# img = cv2.imread(image_path)
# #     print(image_path)
# output = !./darknet detector test build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/yolo-obj_2000.weights -ext_output $image_path -dont_show
# fig, ax = plt.subplots(1, 1, figsize=(16, 8))
# for line in output:
#     if re.search(PAT, line):
#         arrtemp = line.split()
#         left_x = int(arrtemp[3])
#         if left_x<0:
#             left_x = 0
#         top_y = int(arrtemp[5])
#         if top_y<0:
#             top_y = 0
#         xmin = np.int(left_x)
#         ymin = np.int(top_y)
#         width = np.int(arrtemp[7])
#         height = np.int(arrtemp[9][:-1])
#         img = cv2.rectangle(img,(xmin,ymin),(xmin+width,ymin+height),(255,0,0),2)
# ax.set_axis_off()
# ax.imshow(img)

In [ ]:
# %ls
!cp submission.csv /kaggle/working
os.chdir("/kaggle/working")
!rm -rf yolowheatdetectionnoopencv

In [ ]:
# from PIL import Image, ImageEnhance 
# im = Image.open("wheat3.jpg")
# enhancer = ImageEnhance.Brightness(im)
# enhanced_im = enhancer.enhance(1)
# enhanced_im.save("enhanced.sample5.png")

In [ ]:
# !./darknet detector test build/darknet/x64/data/obj.data build/darknet/x64/cfg/yolo-obj.cfg build/darknet/x64/yolo-obj_2000.weights -ext_output color_img.jpg -dont_show


In [ ]:
# image = cv2.imread("wheat3.jpg")

# new_image = np.zeros(image.shape, image.dtype)
# alpha = 1.3 # Simple contrast control
# beta =40    # Simple brightness control
# # Initialize values
# # alpha = float(input('* Enter the alpha value [1.0-3.0]: '))
# # beta = int(input('* Enter the beta value [0-100]: '))
# # Do the operation new_image(i,j) = alpha*image(i,j) + beta
# # Instead of these 'for' loops we could have used simply:
# # new_image = cv.convertScaleAbs(image, alpha=alpha, beta=beta)
# # but we wanted to show you how to access the pixels :)
# for y in range(image.shape[0]):
#     for x in range(image.shape[1]):
#         for c in range(image.shape[2]):
#             new_image[y,x,c] = np.clip(alpha*image[y,x,c] + beta, 0, 255)
# fig, ax = plt.subplots(1, 1, figsize=(16, 8))
# # cv2.imshow('Original Image', image)
# # cv2.imshow('New Image', new_image)
# ax.set_axis_off()
# ax.imshow(image)
# # ax.imshow(new_image)
# cv2.imwrite('color_img.jpg', image)
# # new_image.save("enhanced.sample5.png")

In [ ]:
# def adjust_gamma(image, gamma=1.0):
# # build a lookup table mapping the pixel values [0, 255] to
# # their adjusted gamma values
#     invGamma = 1.0 / gamma
#     table = np.array([((i / 255.0) ** invGamma) * 255
#     for i in np.arange(0, 256)]).astype("uint8")
# # apply gamma correction using the lookup table
#     return cv2.LUT(image, table)

# original = cv2.imread("wheat3.jpg")
# adjusted = adjust_gamma(original, 0.5)
# fig, ax = plt.subplots(1, 1, figsize=(16, 8))
# # cv2.imshow('Original Image', image)
# # cv2.imshow('New Image', new_image)
# ax.set_axis_off()
# ax.imshow(adjusted)
# # ax.imshow(new_image)
# cv2.imwrite('color_img.jpg', adjusted)